# Data Analysis and Schema Generation with TFDV

In this lab, we use [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv) (TFDV) to perform the following:

1. **Generate statistics** from the training data.
2. **Visualise and analyse** the generated statistics.
2. **Infer** a **schema** from the generated statistics.
3. **Update** the schema with domain knowledge.
4. **Validate** the evaluation data against the schema.
5. **Save** the schema for later use.

![alt text](imgs/tfdv.png "Data analysis and schema generation")


In [3]:
#!pip install -q -U tensorflow_data_validation

In [6]:
import os
import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.__version__))

TFDV version: 0.15.0


In [13]:
WORKSPACE = 'workspace' # you can set to a GCS location
DATA_DIR = os.path.join(WORKSPACE, 'raw_data')
TRAIN_DATA_FILE = os.path.join(DATA_DIR,'train.csv')
EVAL_DATA_FILE = os.path.join(DATA_DIR,'eval.csv')

## 1. Generate statistics

In [14]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
               'marital_status', 'occupation', 'relationship', 'race', 'gender',
               'capital_gain', 'capital_loss', 'hours_per_week',
               'native_country', 'income_bracket']

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

You can run this on Dataflow by setting the `pipeline_options` parameter.

In [15]:
train_stats = tfdv.generate_statistics_from_csv(
    data_location=TRAIN_DATA_FILE, 
    column_names=HEADER,
    stats_options=tfdv.StatsOptions(
        weight_feature=WEIGHT_COLUMN_NAME,
        sample_rate=1.0
    )
)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## 2. Visualise generated statistics

In [19]:
tfdv.visualize_statistics(train_stats)

## 3. Infer schema from statistics

In [23]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'income_bracket',STRING,required,,'income_bracket'
'occupation',STRING,required,,'occupation'
'capital_loss',INT,required,,-
'marital_status',STRING,required,,'marital_status'
'relationship',STRING,required,,'relationship'
'education',STRING,required,,'education'
'capital_gain',INT,required,,-
'fnlwgt',INT,required,,-
'hours_per_week',INT,required,,-


,Values
Domain,
'income_bracket',"' <=50K', ' >50K'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'gender',"' Female', ' Male'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'native_country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


## 4. Update the schema with yout domain knowledge

In [24]:
# Relax the minimum fraction of values that must come from the domain for feature occupation.
occupation = tfdv.get_feature(schema, 'occupation')
occupation.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature native_country.
native_country_domain = tfdv.get_domain(schema, 'native_country')
native_country_domain.value.append('Egypt')

# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('EVALUATION')
schema.default_environment.append('SERVING')

# Specify that the class feature is not in SERVING environment.
tfdv.get_feature(schema, TARGET_FEATURE_NAME).not_in_environment.append('SERVING')

In [26]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'income_bracket',STRING,required,,'income_bracket'
'occupation',STRING,required,,'occupation'
'capital_loss',INT,required,,-
'marital_status',STRING,required,,'marital_status'
'relationship',STRING,required,,'relationship'
'education',STRING,required,,'education'
'capital_gain',INT,required,,-
'fnlwgt',INT,required,,-
'hours_per_week',INT,required,,-


,Values
Domain,
'income_bracket',"' <=50K', ' >50K'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'gender',"' Female', ' Male'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'native_country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', 'Egypt'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


In [27]:
tfdv.get_feature(schema, TARGET_FEATURE_NAME)

name: "income_bracket"
type: BYTES
domain: "income_bracket"
presence {
  min_fraction: 1.0
  min_count: 1
}
not_in_environment: "SERVING"
shape {
  dim {
    size: 1
  }
}

## 4. Validate evaluation data
We validate evaluation data against the generated schema, and find anomalies, if any...

In [37]:
eval_stats = tfdv.generate_statistics_from_csv(
    EVAL_DATA_FILE, 
    column_names=HEADER, 
    stats_options=tfdv.StatsOptions(
        weight_feature=WEIGHT_COLUMN_NAME)
)

eval_anomalies = tfdv.validate_statistics(eval_stats, schema, environment='EVALUATION')
tfdv.display_anomalies(eval_anomalies)

## 5. Save the schema
We freeze the schema to use it for the subsequent ML steps.

In [32]:
RAW_SCHEMA_LOCATION = os.path.join(WORKSPACE, 'raw_schema.pbtxt')

In [33]:
tfdv.write_schema_text(schema, RAW_SCHEMA_LOCATION)
print("Schema stored.")

Schema stored.


In [34]:
tfdv.load_schema_text(RAW_SCHEMA_LOCATION)

feature {
  name: "income_bracket"
  type: BYTES
  domain: "income_bracket"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  not_in_environment: "SERVING"
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "occupation"
  type: BYTES
  domain: "occupation"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  distribution_constraints {
    min_domain_mass: 0.9
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "capital_loss"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "marital_status"
  type: BYTES
  domain: "marital_status"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "relationship"
  type: BYTES
  domain: "relationship"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "education"
  type: BYTES
  domain: "educatio